In [1]:
import pandas as pd 
from matplotlib import pyplot as plt 
import numpy as np 
import matplotlib 
from sqlalchemy import create_engine
from sshtunnel import SSHTunnelForwarder

%matplotlib inline 

In [2]:
''' 
bitconnect to the database 
'''
def get_db_connection():
    username = 'mig'
    password = 'password' 
    host_name = 'localhost'
    port = 5432
    db_name = 'mtg'
    conn_str = 'postgresql://{}:{}@{}:{}/{}'.format(username, password, host_name, port, db_name)

    engine = create_engine(conn_str)
    return engine

In [3]:
def get_db_connection():
    
    '''
    close connections by doing 

    engine.dispose()
    server.close()
    '''
    server = SSHTunnelForwarder(
         ('mtgdata.ddns.net', 49000),
         ssh_password="red",
         ssh_username="mig",
         remote_bind_address=('127.0.0.1', 5432)
    )

    server.start()

    username = 'mig'
    password = 'password' 
    host_name = 'localhost'
    port = server.local_bind_port # assigned local port
    db_name = 'mtg'
    conn_str = 'postgresql://{}:{}@{}:{}/{}'.format(username, password, host_name, port, db_name)

    engine = create_engine(conn_str)
    
    return engine, server

In [4]:
#fix the ts
def correct_ts(ts):
    minute = 0 if ts.minute < 30 else 30
    ts_corrected = pd.Timestamp(ts.year, ts.month, ts.day, ts.hour, minute)
    return ts_corrected

def data_transformer(df):
    df = df.copy()
    df.ts = df.ts.map(correct_ts)
    df = df.drop('list_order', axis=1)
    df = df.set_index(['ts', 'card_name'])
    df = df.sort_index(level=['ts', 'card_name'])
    return df

In [5]:
'''
read the records
'''

now = pd.Timestamp.now(tz='UTC').date()
now_minus_24_hours = now - pd.Timedelta('%d hours'%(24,))
engine, server = get_db_connection()

query = '''
SELECT *  
FROM card_listings;
'''#%(str(now_minus_24_hours), str(now))

df_db = pd.read_sql_query(query, engine)
engine.dispose()
server.close()

2019-10-15 10:30:53,424| ERROR   | Could not resolve IP address for mtgdata.ddns.net, aborting!


BaseSSHTunnelForwarderError: Could not establish session to SSH gateway

In [ ]:
df_db.dtypes

In [ ]:
df_db.shape

In [ ]:
df_db.card_name.value_counts() 

In [ ]:
df = data_transformer(df_db)
idx = pd.IndexSlice
#the dates before '2019-10-10 23:00:00' have bad data, lets remove them
df = df.loc[idx['2019-10-10 23:00:00':, :], :]

In [ ]:
df.item_location.index

In [ ]:
df.shape

In [ ]:
df.groupby(level=['card_name', 'ts']).item_amount.sum()

In [ ]:
df.loc[idx[:, :], 'item_amount']

In [ ]:
df_xs = df.xs('Snow Covered Island', level='card_name', drop_level=True)

In [ ]:
df_xs.loc['2019-10-13 13:30:00', 'item_amount'].sum()

In [ ]:
df_plot = df_xs.loc[:'2019-10-10 23:00:00', 'item_amount']
df_plot.groupby(df_plot.index).sum().plot(figsize=(16, 4), rot=20); 